### The objective of this project is to create the Team of the Tournament for each edition of IPL

In [1]:
import os
import numpy as np
import pandas as pd

# For scaling Player Stats
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

In [2]:
path_matches = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Data\matches.csv")
path_deliveries = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Data\deliveries.csv")

matches = pd.read_csv(path_matches)
balls = pd.read_csv(path_deliveries)
print(matches.shape, balls.shape)

(1095, 20) (260920, 17)


In [3]:
matches.head(2)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri


In [4]:
balls.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


### To create the Team of Tournament - we need the records and statistics of each player for each of the IPL edition

In [5]:
seasons = matches['season'].unique()
print(seasons)

['2007/08' '2009' '2009/10' '2011' '2012' '2013' '2014' '2015' '2016'
 '2017' '2018' '2019' '2020/21' '2021' '2022' '2023' '2024']


#### FUNCTION TO INSERT PLAYER RECORDS SEASON WISE

In [6]:
def insertRecords(season):
    global data, matches, balls
    matches_played = {}
    
    # CREATE DATAFRAME FOR EACH SEASON AND FIRST LETS INSERT - PLAYER_NAME, TEAM, SEASON, BATTING_POSITION, INNINGS_PLAYED
    cols = ['player_name', 'team', 'season', 'innings_played', 'batting_position', 
            'runs_scored', 'balls_faced', 'dismissals', 'fours', 'sixes', 
            'wickets', 'runs_conceded', 'balls_bowled']
    data[season] = pd.DataFrame(columns=cols)
    
    #####----------------------------------------- PLAYER INFO ------------------------------------------------------###############
    match_ids = matches[matches['season']==season]['id'].values
    for match_number, match_id in enumerate(match_ids):
        
        for i in list(balls[balls['match_id']==match_id]['inning'].unique()): ## Iterate over innings
            batters = list(balls[(balls['match_id']==match_id) & (balls['inning']==i)]['batter'].unique())
            non_strikers = list(balls[(balls['match_id']==match_id) & (balls['inning']==i)]['non_striker'].unique())
            bowlers = list(balls[(balls['match_id']==match_id) & (balls['inning']==i)]['bowler'].unique())
            
            # Trick
            batters = batters + non_strikers
            non_strikers = non_strikers + batters
            
            batting_lineup = []
            for batter, non_striker in zip(batters, non_strikers):
                if batter not in batting_lineup:
                    batting_lineup.append(batter)
                if non_striker not in batting_lineup:
                    batting_lineup.append(non_striker)
        
            batting_team = balls[(balls['match_id'] == match_id) & (balls['inning']==i)]['batting_team'].iloc[0]
            bowling_team = balls[(balls['match_id'] == match_id) & (balls['inning']==i)]['bowling_team'].iloc[0]
            
            # ADDING BATTERS TO SEASON DATAFRAME
            for pos, player in enumerate(batting_lineup):
                if pos in [0, 1]: pos = 'open'
                elif pos in [2, 3]: pos='top'
                else: pos='notRequired'
                
                flag = ((data[season]['player_name']==player) & (data[season]['batting_position']==pos)).any()
                if not flag:
                    new_data = pd.DataFrame([{
                        'player_name': player, 'team':batting_team, 
                        'season':season, 'innings_played':1, 'batting_position': pos,
                        'runs_scored':0, 'balls_faced':0, 'dismissals':0, 'fours':0, 'sixes':0,
                        'wickets':0, 'runs_conceded':0, 'balls_bowled':0
                        }])
                    data[season] = pd.concat([data[season], new_data], ignore_index=True)
                else:
                    data[season].loc[(data[season]['player_name'] == player) & (data[season]['batting_position'] == pos), 'innings_played'] += 1
                
            # Adding bowlers
            for pos, player in enumerate(bowlers):
                pos = 'notRequired'
                flag = ((data[season]['player_name']==player) & (data[season]['batting_position']==pos)).any()
                if not flag:
                    new_data = pd.DataFrame([{
                        'player_name': player, 'team':bowling_team, 
                        'season':season, 'innings_played':1, 'batting_position': pos,
                        'runs_scored':0, 'balls_faced':0, 'dismissals':0, 'fours':0, 'sixes':0,
                        'wickets':0, 'runs_conceded':0, 'balls_bowled':0
                        }])
                    data[season] = pd.concat([data[season], new_data], ignore_index=True)
                else:
                    data[season].loc[(data[season]['player_name'] == player) & (data[season]['batting_position'] == pos), 'innings_played'] += 1
        
    ##--------------------------------------------- RECORDS ---------------------------------------------------------############
    # Now iterate through each ball to update player records
    for match_num, match_id in enumerate(match_ids):
        # print(f"This is {match_num} btw {matches[matches['id']==match_id]['team1'].iloc[0]} and {matches[matches['id']==match_id]['team2'].iloc[0]}")
        for inning in list(balls[balls['match_id']==match_id]['inning'].unique()): ## Iterate over innings
            
            batters = list(balls[(balls['match_id']==match_id) & (balls['inning']==inning)]['batter'].unique())
            non_strikers = list(balls[(balls['match_id']==match_id) & (balls['inning']==inning)]['non_striker'].unique())
            # bowlers = list(balls[(balls['match_id']==match_id) & (balls['inning']==inning)]['bowler'].unique())
            
            # Trick
            batters = batters + non_strikers
            non_strikers = non_strikers + batters
            
            batting_lineup = []
            for batter, non_striker in zip(batters, non_strikers):
                if batter not in batting_lineup:
                    batting_lineup.append(batter)
                if non_striker not in batting_lineup:
                    batting_lineup.append(non_striker)
            
            batting_team = balls[(balls['match_id'] == match_id) & (balls['inning']==inning)]['batting_team'].iloc[0]
            bowling_team = balls[(balls['match_id'] == match_id) & (balls['inning']==inning)]['bowling_team'].iloc[0]
            
            player_pos = {}
            for i, player in enumerate(batting_lineup): 
                if i in [0, 1]: player_pos[player] = 'open'
                elif i in [2, 3]: player_pos[player] = 'top'
                else: player_pos[player]= 'notRequired'
            
            for index, row in balls[(balls['match_id']==match_id) & (balls['inning']==inning)].iterrows():
                
                flag_batter = (data[season]['player_name']==row['batter']) & (data[season]['batting_position']==player_pos[row['batter']])
                # Update total runs scored by the batter
                data[season].loc[flag_batter, 'runs_scored'] += row['batsman_runs']
                
                # Update Fours
                if row['batsman_runs'] == 4:
                    data[season].loc[flag_batter, 'fours'] += 1
                
                # Update Sixes
                if row['batsman_runs'] == 6:
                    data[season].loc[flag_batter, 'sixes'] += 1
                    
                # Update balls_faced by the batsman
                if row['extras_type'] != 'wides':
                    data[season].loc[flag_batter, 'balls_faced'] += 1
##### Dismissal Kinds
# ['caught', 'bowled', 'run out', 'lbw', 'caught and bowled', 'stumped', 'retired hurt', 'hit wicket', 'obstructing the field', 'retired out']
                # Update dismissals
                if row['is_wicket']==1:
                    flag_dismissed = (data[season]['player_name']==row['player_dismissed']) & (data[season]['batting_position']==player_pos[row['player_dismissed']])
                    if row['dismissal_kind'] not in ['retired hurt']:
                        data[season].loc[flag_dismissed, 'dismissals'] += 1
                    
                # Update Wickets
                flag_bowler = (data[season]['player_name']==row['bowler']) & (data[season]['batting_position']=='notRequired')
                if row['is_wicket']==1:
                    if row['dismissal_kind'] in ['caught', 'bowled', 'lbw', 'caught and bowled', 'stumped', 'hit wicket']:
                        data[season].loc[flag_bowler, 'wickets'] += 1
                
                # Update Runs Conceded (Hint: Exclude Legbyes and Byes)
                if row['extras_type'] in ['legbyes', 'byes']:
                    pass
                else:
                    data[season].loc[flag_bowler, 'runs_conceded'] += row['total_runs']
                
                # Update Balls Bowled
                if row['extras_type'] in ['legbyes', 'byes', np.nan]:
                    data[season].loc[flag_bowler, 'balls_bowled'] += 1
                
    print(f"Season {season} is done!!")

In [7]:
# # USING INSERT FUNCTION FOR ALL SEASONS
data = {}
for season in seasons:
    insertRecords(season)
    # break

Season 2007/08 is done!!
Season 2009 is done!!
Season 2009/10 is done!!
Season 2011 is done!!
Season 2012 is done!!
Season 2013 is done!!
Season 2014 is done!!
Season 2015 is done!!
Season 2016 is done!!
Season 2017 is done!!
Season 2018 is done!!
Season 2019 is done!!
Season 2020/21 is done!!
Season 2021 is done!!
Season 2022 is done!!
Season 2023 is done!!
Season 2024 is done!!


In [137]:
# Combine all season data into single dataframe - df
df = pd.DataFrame()
for season in seasons:
    df = pd.concat([df, data[season]], ignore_index=True)
# path_folder = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code")
# df.to_excel(path_folder + 'iplPlayers.xlsx')
# file_path = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code\iplPlayers.xlsx")
# df = pd.read_excel(file_path)
print(df.shape)
df.head(3)

(4095, 13)


,player_name,team,season,innings_played,batting_position,runs_scored,balls_faced,dismissals,fours,sixes,wickets,runs_conceded,balls_bowled
0,SC Ganguly,Kolkata Knight Riders,2007/08,6,open,168,132,5,15,8,0,0,0
1,BB McCullum,Kolkata Knight Riders,2007/08,4,open,188,92,3,13,15,0,0,0
2,RT Ponting,Kolkata Knight Riders,2007/08,4,top,39,53,4,2,1,0,0,0


In [138]:
# Changing datatypes
df[['player_name', 'team', 'batting_position']] = df[['player_name', 'team', 'batting_position']].astype(str)
df[['innings_played', 'runs_scored', 'balls_faced', 'dismissals', 'fours', 
        'sixes', 'wickets', 
        'runs_conceded', 'balls_bowled']] = df[['innings_played', 
                                                'runs_scored', 'balls_faced', 'dismissals', 
                                                'fours', 'sixes', 'wickets', 'runs_conceded', 'balls_bowled']].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4095 entries, 0 to 4094
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   player_name       4095 non-null   object
 1   team              4095 non-null   object
 2   season            4095 non-null   object
 3   innings_played    4095 non-null   int32 
 4   batting_position  4095 non-null   object
 5   runs_scored       4095 non-null   int32 
 6   balls_faced       4095 non-null   int32 
 7   dismissals        4095 non-null   int32 
 8   fours             4095 non-null   int32 
 9   sixes             4095 non-null   int32 
 10  wickets           4095 non-null   int32 
 11  runs_conceded     4095 non-null   int32 
 12  balls_bowled      4095 non-null   int32 
dtypes: int32(9), object(4)
memory usage: 272.1+ KB


In [169]:
def insertStats(mat):
    # INSERT BATTING AVERAGE
    mat['batting_average'] = mat.apply(lambda row: row['runs_scored']/row['dismissals'] 
                                                if row['dismissals']>0 else row['runs_scored'], axis=1)
    # INSERT BATTING STRIKE RATE
    mat['batting_strike_rate'] = mat.apply(lambda row: 100*(row['runs_scored']/row['balls_faced'])
                                                if row['balls_faced']>0 else 0, axis=1)
    # INSERT BOWLING AVERAGE
    mat['bowling_average'] = mat.apply(lambda row: row['runs_conceded']/row['wickets']
                                             if row['wickets']>0 else 100, axis=1)
    # INSERT BOWLING STRIKE RATE
    mat['bowling_strike_rate'] = mat.apply(lambda row: row['balls_bowled']/row['wickets']
                                                 if row['wickets']>0 else 100, axis=1)
    # INSERT BOWLING ECONOMY RATE
    mat['economy_rate'] = mat.apply(lambda row: row['runs_conceded']/(row['balls_bowled']/6)
                                          if (row['balls_bowled']/6)>0 else 100, axis=1)
    
    return mat

def addBestPlayers(lst, season, start, end):
    global best11s
    for player in lst:
        if start>end: break
        if player not in list(best11s.loc[:, season]):
            best11s.loc[start, season] = player
            start += 1

def getScore(mat, key):
    mat = mat[['player_name', 'runs_scored', 'batting_average', 'batting_strike_rate', 'fours', 'sixes', 
               'wickets', 'bowling_average', 'bowling_strike_rate', 'economy_rate']]
    mat = mat.set_index('player_name')
    # Scaling the stats
    mat[['runs_scored', 'batting_average', 
         'batting_strike_rate', 'fours', 'sixes', 
               'wickets', 'bowling_average', 
               'bowling_strike_rate', 'economy_rate']] = MinMaxScaler().fit_transform(mat[['runs_scored', 'batting_average', 
                                                                                           'batting_strike_rate', 'fours', 'sixes', 
                                                                                           'wickets', 'bowling_average', 
                                                                                           'bowling_strike_rate', 'economy_rate']])
    mat[key] = mat.dot(params.loc[key])
    mat = mat.sort_values(by=key, ascending=False).reset_index()
    return mat

# Parameters for statistics
params = pd.DataFrame(index=['open','top', 'batAll', 'bowlAll', 'bowler'], columns=['runs_scored', 
                                                                                    'batting_average', 'batting_strike_rate', 
                                                                                    'fours', 'sixes', 'wickets', 'bowling_average', 
                                                                                    'bowling_strike_rate', 'economy_rate'])
params.loc['open'] = [0.6, 0.1, 0.2, 0.05, 0.05, 0, 0, 0, 0]
params.loc['top'] = [0.6, 0.1, 0.2, 0.05, 0.05, 0, 0, 0, 0]
params.loc['batAll'] = [0.6, 0.4, 0.3, 0.1, 0.1, 0.4, -0.2, -0.2, -0.2]
params.loc['bowlAll'] = [0.3, 0.25, 0.2, 0.1, 0.1, 0.7, -0.3, -0.3, -0.3]
params.loc['bowler'] = [0, 0, 0, 0, 0, 0.9, -0.4, -0.4, -0.4]

# Preparing dataframe for storing team of the tournaments
best11s = pd.DataFrame(columns=seasons, index=list(range(1, 12)))
best11s[:] = 0

for season in seasons:
    # if not season == '2011': continue
    global df
    
    ##----------------------------------------------------------------------------##########################-------------------
    # Getting Openers (2)
    mat = df[(df['season']==season) & (df['batting_position']=='open')].drop(columns=['season', 'innings_played', 
                                                                                      'batting_position', 
                                                                                      'team']).groupby('player_name').sum().reset_index()
    mat = insertStats(mat)
    mat = getScore(mat, 'open')
    mat = mat.head(2)
    openers = list(mat['player_name'])
    addBestPlayers(openers, season, 1, 2)
    del mat
    
    # ##----------------------------------------------------------------------------##########################-------------------
    # # Getting Top Order (2)
    mat = df[(df['season']==season) & (df['batting_position']=='top')].drop(columns=['team', 'season', 'innings_played', 
                                                                    'batting_position']).groupby('player_name').sum().reset_index()
    mat = insertStats(mat)
    mat = getScore(mat, 'top')
    # mat = mat.reset_index().sort_values(by='top', ascending=False).head(4)
    mat = mat.head(4)
    topOrder = list(mat['player_name'])
    addBestPlayers(topOrder, season, 3, 4)
    del mat
    
    # # ##----------------------------------------------------------------------------##########################-------------------
    # # # Getting 3 Bowlers
    mat = df[df['season']==season].drop(columns=['team', 'season', 'innings_played', 
                                                   'batting_position']).groupby('player_name').sum().reset_index()
    
    mat = insertStats(mat)
    mat = getScore(mat, 'bowler')
    mat = mat.reset_index().sort_values(by='bowler', ascending=False).head(11)
    bowlers = list(mat['player_name'])
    addBestPlayers(bowlers, season, 9, 11)
    best11s.loc[[9, 10, 11], season] = best11s.loc[[9, 10, 11], season][::-1].values
    del mat
    
    # # ##----------------------------------------------------------------------------##########################-------------------
    # # # Getting 2 Batting All-Rounders
    mat = df[df['season']==season].drop(columns=['team', 'season', 'innings_played', 
                                                   'batting_position']).groupby('player_name').sum().reset_index()
    
    mat = insertStats(mat)
    mat = getScore(mat, 'batAll')
    mat = mat.reset_index().sort_values(by='batAll', ascending=False).head(6)
    battingallRounders = list(mat['player_name'])
    addBestPlayers(battingallRounders, season, 5, 6)
    del mat
    
    # # ##----------------------------------------------------------------------------##########################-------------------
    # # # Getting 2 Bowling All-Rounders
    mat = df[df['season']==season].drop(columns=['team', 'season', 'innings_played', 
                                                   'batting_position']).groupby('player_name').sum().reset_index()
    
    mat = insertStats(mat)
    mat = getScore(mat, 'bowlAll')
    mat = mat.reset_index().sort_values(by='bowlAll', ascending=False).head(8)
    bowlingallRounders = list(mat['player_name'])
    addBestPlayers(bowlingallRounders, season, 7, 8)
    # Reverse the Order of Bowlers
    best11s.loc[[7, 8], season] = best11s.loc[[7, 8], season][::-1].values
    del mat

In [170]:
best11s.head(11)

,2007/08,2009,2009/10,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020/21,2021,2022,2023,2024
1,SE Marsh,ML Hayden,SR Tendulkar,CH Gayle,CH Gayle,CH Gayle,RV Uthappa,DA Warner,V Kohli,DA Warner,KL Rahul,DA Warner,KL Rahul,KL Rahul,JC Buttler,Shubman Gill,V Kohli
2,ST Jayasuriya,AC Gilchrist,JH Kallis,SR Tendulkar,G Gambhir,MEK Hussey,DR Smith,LMP Simmons,DA Warner,S Dhawan,SR Watson,KL Rahul,S Dhawan,RD Gaikwad,KL Rahul,F du Plessis,TM Head
3,SR Watson,AB de Villiers,SK Raina,V Kohli,CL White,V Kohli,GJ Maxwell,AB de Villiers,AB de Villiers,SPD Smith,RR Pant,AB de Villiers,SS Iyer,SV Samson,HH Pandya,SA Yadav,R Parag
4,SK Raina,SK Raina,RV Uthappa,SE Marsh,SK Raina,SK Raina,SK Raina,JP Duminy,SK Raina,SK Raina,KS Williamson,SS Iyer,SA Yadav,GJ Maxwell,SV Samson,C Green,SV Samson
5,YK Pathan,RG Sharma,A Symonds,PC Valthaty,S Dhawan,SR Watson,LMP Simmons,AD Russell,CH Morris,GJ Maxwell,SP Narine,AD Russell,MP Stoinis,RA Jadeja,AD Russell,GJ Maxwell,SP Narine
6,JA Morkel,BJ Hodge,KA Pollard,JH Kallis,DJ Bravo,KA Pollard,Yuvraj Singh,KA Pollard,YK Pathan,BA Stokes,AD Russell,HH Pandya,KA Pollard,VR Iyer,LS Livingstone,MP Stoinis,T Stubbs
7,MF Maharoof,MM Patel,Z Khan,A Mishra,KA Pollard,Harbhajan Singh,VR Aaron,Harbhajan Singh,DS Kulkarni,P Negi,SN Thakur,RA Jadeja,A Nortje,AD Russell,Rashid Khan,PP Chawla,SM Curran
8,PP Chawla,IK Pathan,IK Pathan,YK Pathan,JH Kallis,R Vinay Kumar,RA Jadeja,MC Henriques,AD Russell,AR Patel,HH Pandya,S Gopal,JC Archer,DJ Bravo,Kuldeep Yadav,RA Jadeja,AD Russell
9,S Sreesanth,A Nehra,Harbhajan Singh,S Aravind,SL Malinga,MG Johnson,B Kumar,YS Chahal,SR Watson,JJ Bumrah,S Kaul,DL Chahar,TA Boult,JJ Bumrah,K Rabada,Rashid Khan,JJ Bumrah
10,SK Warne,A Kumble,A Mishra,MM Patel,SP Narine,JP Faulkner,SP Narine,SL Malinga,YS Chahal,JD Unadkat,Rashid Khan,Imran Tahir,JJ Bumrah,Avesh Khan,PWH de Silva,MM Sharma,CV Varun


In [171]:
# file_path = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\IPLTeamOfTournaments\Code")
# file_name = os.path.join(file_path, 'allstar.xlsx')
# best11s.to_excel(file_name, index=False)